# **Задачи**
**1)** Создать декоратор, который будет запускать функцию в отдельном
потоке. Декоратор должен принимать следующие аргументы:
название потока, является ли поток демоном.

**Здравствуйте, замечания по 5 занятию.

1) Задача
1. Количество аргументов во враппере фиксированное, а это значит что декоратор будет корректно отрабатывать только для Вашей конкретной функции, а функции с другим кол-вом аргументов, он не сможет воспроизвести.
2 Враппер не возвращает никакого результата**

In [0]:
#на Colab почему то не отрабатывает   print('End thread\n') в функции, которую мы пускаем 
#через поток. Долго мучался, так пока что не понял, как это пофиксить.
#в PyCharm все нормально отрабатывает....

import time
from threading import Thread

def thread_decorator_settings(thread_name='two', daemon_flag=True):

  def decorator(func):
    
    def wrapper(sec):
      target = Thread(target=func, name=thread_name, args=(sec, ), daemon=daemon_flag )
      target.start()
      print(f"Our Thread Name Is:{target.getName()}, and DaemonStatus:{target.isDaemon()}")
    
    return wrapper
  return decorator


@thread_decorator_settings(thread_name='one', daemon_flag=False)
def hello_thread(sec):
  print('Start thread\n')
  time.sleep(sec)
  print('End thread\n')


print('Before decorator function\n')
hello_thread(15)
print("After decorator function\n")


Before decorator function

Start thread
Our Thread Name Is:one, and DaemonStatus:False
After decorator function




**2)** Создать функцию, которая будет скачивать файл из интернета по
ссылке, повесить на неё созданный декоратор. Создать список из 10
ссылок, по которым будет происходить скачивание. Создать список
потоков, отдельный поток, на каждую из ссылок. Каждый поток
должен сигнализировать, о том, что, он начал работу и по какой
ссылке он работает, так же должен сообщать когда скачивание
закончится.

**2) задача
1. Как написано у Вас комментах, что проигнорированы примитивы, и Ваши операции не атомарны, что ни есть хорошо. 
2. Зачем использоывать цикл while и индексы, когда можно было бы функции запускать по каждой ссылке с помощью цикла фор
3. Глобальные переменные зло, используйте их для констант
4. Опять же декоратор заточен чисто под один аргумент
5. А Вы то потоки сразу джоинили, так что думаю файлы все таки файлы у Вас не паралельно скачивались, а последовательно.**

In [0]:
#Маунтим диск в Colab
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#Создаем директорию для записи файлов
!mkdir "/content/drive/My Drive/downloadtest"
%cd /content/drive/My Drive/downloadtest/
!ls

mkdir: cannot create directory ‘/content/drive/My Drive/downloadtest’: File exists
/content/drive/My Drive/downloadtest
i1040gi.pdf


In [0]:
#Скрипт работает следующим образом.
#Передаем в переменную урл сайта, по которому хотим с переданной нам страницы скачать
#Все pdf документы. Ну а дальше парсим страницу и в 10 потоков качаем все доки на 
#этой странице. Реализовывал одновременное количество потоков без семафоров и замков.
#Сделано рукожопно (. Осознаю. 

import time
import threading
from threading import Thread
import requests, re


num_threads = 10
count = 0

def decorator(func):
  
  def wrapper(file_link):
    global count
    number_files = len(parsed_pdf_links)
    print("number_files",number_files)
    print(f'Number of active threads:{threading.active_count()}')
    
    while count < number_files:#number_files
      
      if threading.active_count() < num_threads + 1 :
        count +=1
        thread_name = f"thread{count}:"
        file_link = parsed_pdf_links[count - 1]
        thread = Thread(target=func, args=(file_link, ), name=thread_name)
        thread.start()
        
        print(f'Number of active threads:{threading.active_count()}')
        print(f'{threading.currentThread().getName()} started')
        thread.join()
      
      print(f"{count}:{file_link}\n")

    
  return wrapper    



@decorator
def pdf_downloader(file_link):
    global count
    file_name = re.findall(r"/(\w+\.pdf)", file_link)
    result = requests.get(file_link, allow_redirects=True)
   
    with open(file_name[0], 'wb') as f:
        f.write(result.content)
    print(f'{threading.currentThread().getName()} closed')
    #count += 1
    print(f'Number of active threads:{threading.active_count()}')
    


our_url = "http://www.irs.gov/pub/irs-pdf/"
pattern = r"https://www.irs.gov/pub/irs-pdf/\w+.pdf"
result = requests.get(our_url)
parsed_pdf_links = re.findall(pattern, result.text)
  

if parsed_pdf_links:
      pdf_downloader(file_link)   


number_files 52
Number of active threads:5
Number of active threads:6
MainThread started
thread1: closed
Number of active threads:6
1:https://www.irs.gov/pub/irs-pdf/i1040gi.pdf

Number of active threads:6
MainThread started
thread2: closed
Number of active threads:6
2:https://www.irs.gov/pub/irs-pdf/i1040gi.pdf

Number of active threads:6
MainThread started
thread3: closed
Number of active threads:6
3:https://www.irs.gov/pub/irs-pdf/p2104b.pdf

Number of active threads:6
MainThread started
thread4: closed
Number of active threads:6
4:https://www.irs.gov/pub/irs-pdf/p5286.pdf

Number of active threads:6
MainThread started
thread5: closed
Number of active threads:6
5:https://www.irs.gov/pub/irs-pdf/p1546ezh.pdf

Number of active threads:6
MainThread started
thread6: closed
Number of active threads:6
6:https://www.irs.gov/pub/irs-pdf/p2104.pdf

Number of active threads:6
MainThread started
thread7: closed
Number of active threads:6
7:https://www.irs.gov/pub/irs-pdf/p5412k.pdf

Number of 